In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fide-google-efficiency-chess-ai-challenge/Screenshot 2024-10-09 at 10.45.28AM.png


In [4]:
!pip install chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.1-py3-none-any.whl size=148498 sha256=09575174763e71a0a343b34ae7d0016e38ac99979a453315eb56fbb3c57f1ba8
  Stored in directory: /root/.cache/pip/wheels/2e/2d/23/1bfc95db984ed3ecbf6764167dc7526d0ab521cf9a9852544e
Successfully built chess


In [7]:
import chess
import chess.engine
import time

# Piece values for evaluation
PIECE_VALUES = {
    chess.PAWN: 100,
    chess.KNIGHT: 320,
    chess.BISHOP: 330,
    chess.ROOK: 500,
    chess.QUEEN: 900,
    chess.KING: 0  # King has no value but is vital for the game.
}

# Piece-Square Tables for positional play (simplified to conserve memory)
PAWN_TABLE = [0, 5, 10, 20, 20, 10, 5, 0]
KNIGHT_TABLE = [-10, 0, 10, 20, 20, 10, 0, -10]
EMPTY_TABLE = [0] * 8


def evaluate_board(board):
    """
    Evaluate the board based on material count and piece positioning.
    Positive value is better for White; negative is better for Black.
    """
    value = 0
    for square, piece in board.piece_map().items():
        if piece.color == chess.WHITE:
            value += PIECE_VALUES[piece.piece_type]
            value += positional_bonus(piece.piece_type, square, piece.color)
        else:
            value -= PIECE_VALUES[piece.piece_type]
            value -= positional_bonus(piece.piece_type, square, piece.color)
    return value


def positional_bonus(piece_type, square, color):
    """Bonus points based on piece position."""
    row = chess.square_rank(square) if color == chess.WHITE else 7 - chess.square_rank(square)
    if piece_type == chess.PAWN:
        return PAWN_TABLE[row]
    elif piece_type == chess.KNIGHT:
        return KNIGHT_TABLE[row]
    return 0  # No position bonus for other pieces.


def order_moves(board, legal_moves):
    """
    Order moves to improve alpha-beta pruning efficiency.
    Captures and checks are prioritized.
    """
    ordered = []
    for move in legal_moves:
        board.push(move)
        if board.is_checkmate():
            score = 10000  # Checkmate
        elif board.is_check():
            score = 50  # Checks are valuable
        else:
            score = evaluate_board(board)  # Static evaluation
        ordered.append((score, move))
        board.pop()
    ordered.sort(reverse=True, key=lambda x: x[0])
    return [move for _, move in ordered]


def alpha_beta(board, depth, alpha, beta, maximizing_player, start_time, time_limit):
    """
    Alpha-Beta Pruning with time management.
    """
    if time.time() - start_time > time_limit:
        return evaluate_board(board)  # Return static evaluation on time-out
    if depth == 0 or board.is_game_over():
        return evaluate_board(board)

    legal_moves = list(board.legal_moves)
    ordered_moves = order_moves(board, legal_moves)

    if maximizing_player:
        max_eval = float('-inf')
        for move in ordered_moves:
            board.push(move)
            eval = alpha_beta(board, depth - 1, alpha, beta, False, start_time, time_limit)
            board.pop()
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break  # Beta cutoff
        return max_eval
    else:
        min_eval = float('inf')
        for move in ordered_moves:
            board.push(move)
            eval = alpha_beta(board, depth - 1, alpha, beta, True, start_time, time_limit)
            board.pop()
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break  # Alpha cutoff
        return min_eval


def find_best_move(board, max_depth=4, time_limit=0.1):
    """
    Iterative deepening search to find the best move within a time constraint.
    """
    best_move = None
    start_time = time.time()
    for depth in range(1, max_depth + 1):
        if time.time() - start_time > time_limit:
            break
        legal_moves = list(board.legal_moves)
        ordered_moves = order_moves(board, legal_moves)
        best_eval = float('-inf')
        for move in ordered_moves:
            board.push(move)
            eval = alpha_beta(board, depth, float('-inf'), float('inf'), False, start_time, time_limit)
            board.pop()
            if eval > best_eval:
                best_eval = eval
                best_move = move
        if time.time() - start_time > time_limit:
            break
    return best_move


def main():
    """
    Main loop for the ChessBot.
    """
    board = chess.Board()
    time_limit = 0.1  # 0.1 seconds per move
    while not board.is_game_over():
        print(board)
        if board.turn == chess.WHITE:
            print("White's move...")
            move = find_best_move(board, max_depth=4, time_limit=time_limit)
        else:
            print("Black's move...")
            move = find_best_move(board, max_depth=4, time_limit=time_limit)
        if move:
            print(f"Selected Move: {move}")
            board.push(move)
        else:
            print("No legal moves found.")
            break
    print("Game Over!")
    print("Result:", board.result())


if __name__ == "__main__":
    main()


r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R
White's move...
Selected Move: g1f3
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . N . .
P P P P P P P P
R N B Q K B . R
Black's move...
Selected Move: g7g6
r n b q k b n r
p p p p p p . p
. . . . . . p .
. . . . . . . .
. . . . . . . .
. . . . . N . .
P P P P P P P P
R N B Q K B . R
White's move...
Selected Move: b1a3
r n b q k b n r
p p p p p p . p
. . . . . . p .
. . . . . . . .
. . . . . . . .
N . . . . N . .
P P P P P P P P
R . B Q K B . R
Black's move...
Selected Move: f8h6
r n b q k . n r
p p p p p p . p
. . . . . . p b
. . . . . . . .
. . . . . . . .
N . . . . N . .
P P P P P P P P
R . B Q K B . R
White's move...
Selected Move: g2g4
r n b q k . n r
p p p p p p . p
. . . . . . p b
. . . . . . . .
. . . . . . P .
N . . . . N . .
P P P P P P . P
R . B Q K B . R
Black's move...
Selected Move: h6f8
r n b q k b n r
